In [ ]:
!pip install -Uqq fastai duckduckgo_search gradio

In [ ]:
from fastcore.all import *
from fastai.vision.all import *
from fastai.vision.widgets import *
from fastdownload import download_url
from duckduckgo_search import DDGS
import warnings

warnings.filterwarnings("ignore")

In [ ]:
def search_images(query, max_results=200):
    ddgs = DDGS()
    return L(ddgs.images(query, max_results=max_results)).itemgot('image')

In [ ]:
searches = [
    "Tabby Cat",
    "Ragdoll Cat",
    "British Shorthair Cat",
]

item_number = 1000

arch = resnet34 # resnet50, resnet101, vgg16_bn

In [ ]:
searches = sorted(list(map(lambda x: x.replace(" ", "_"), searches)))

if item_number < 64: raise ValueError("item_number must be at least 64 for batch_size=64")
elif item_number < 200: print("Consider using at least 200 images for better results")

In [ ]:
fig, ax = plt.subplots(1, len(searches), figsize=(5 * len(searches), 5))

examples = []

for i, search in enumerate(searches):
    dest = Path(search + ".jpg")
    examples.append(dest)
    img = PILImage.create(download_url(search_images(search, 1)[0], dest, show_progress=False))
    img.show(ax=ax[i], title=search)

In [ ]:
path = Path("image_classifier_data")

if not path.exists():
    for search in searches:
        dest = path / search
        dest.mkdir(exist_ok=True, parents=True)
        urls=search_images(search, item_number)
        download_images(dest, urls=urls)

failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=0),
    get_y=parent_label,
    item_tfms=Resize(224, method="squish"),
    batch_tfms=aug_transforms()
).dataloaders(path)

dls.show_batch()

In [ ]:
callbacks = [
    SaveModelCallback(monitor="accuracy", fname="best_model"),
    EarlyStoppingCallback(monitor="valid_loss", patience=3)
]

learn = vision_learner(dls, arch, metrics=accuracy, cbs=callbacks)
learn.fine_tune(10)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(10, nrows=2, figsize=(30, 10))

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

In [ ]:
learn.vocab = dls.vocab
learn.examples = examples
learn.export("classifier.pkl")

In [ ]:
import gradio as gr
from fastai.vision.all import *

learn = load_learner("classifier.pkl")

classes = learn.vocab
examples = learn.examples

def classify_image(img):
    pred, idx, probs = learn.predict(img)
    return dict(zip(classes, map(float, probs)))

image = gr.Image()
label = gr.Label(num_top_classes=5)

intf = gr.Interface(fn=classify_image, inputs=image, outputs=label, examples=examples)

intf.launch()

In [ ]:
app_path = Path("app")
app_path.mkdir(exist_ok=True)

shutil.copy("classifier.pkl", app_path / "classifier.pkl")

for example in examples:
    shutil.copy(example, app_path / example)

with open(app_path / "app.py", "w") as f:
    f.write(f"""import gradio as gr
from fastai.vision.all import *

learn = load_learner("classifier.pkl")

classes = learn.vocab
examples = learn.examples

def classify_image(img):
    pred, idx, probs = learn.predict(img)
    return dict(zip(classes, map(float, probs)))

image = gr.Image()
label = gr.Label(num_top_classes=5)

intf = gr.Interface(fn=classify_image, inputs=image, outputs=label, examples=examples)

intf.launch()
""")

with open(app_path / "requirements.txt", "w") as f:
    f.write("fastai\ngradio\ntimm\ntorch\ntorchvision")

shutil.make_archive(app_path, 'zip', app_path)

print("App created successfully!")